# Question: Can the business generate growing FCF and sustain high ROIC?

Return on invested capital is the ultimate measure of business profitability and performance. The ROIC metric is the linchpin that connects a company's sales growth, profitability, free cash flow, and balance sheet.

Here's an example from the McKinsey book Valuation. Assume we have two companies (company A and company B) that aim to grow earnings at a rate of 5% per year, but Company A has a ROIC of 20% and Company B has one of only 10%. Then company A only has to reinvest 25% of its profits to grow earnings 5% (20% x 25% = 5%), but company B has to reinvest 50% of its profits to grow earnings at the same 5% rate (10% x 50% = 5%). The company with the higher ROIC has a lower reinvestment rate, and will need to reinvest less capital to achieve the same level of earnings growth. And because the higher-ROIC business requires less capital (reinvestment) to grow, it generates higher free cash flow.

This means ROIC is the prime driver of FCF and earnings growth. ROIC is not only a measure of quality...but also of growth.

Mauboussin writes: "The rate of return on incremental capital is the maximum growth rate in operating profit a business can reach without external financing." Using the formula ROIC x reinvestment rate = profit growth, we can see that a business that generates a ROIC of 10% can't grow operating profit faster than 10%, and to do so it must reinvest 100% of its profits (10% x 100% = 10%). But a company that generates a 20% ROIC and has opportunities to reinvest 100% of its capital can grow earnings at 20% (20% x 100% =20%).

Because these high-ROIC businesses generate so much FCF, they can finance their growth internally, rather than relying on outside capital to grow. This means less debt (or less equity dilution) for shareholders. It also means that some of the excess FCF can be used to pay down debt, which further strengthens the balance sheet. Common terms used to describe high-ROIC businesses that don't require a lot of capital to grow (and therefore generate strong and growing FCF) are "self-funding," "asset-light," and "compounders." No matter what you call them, ESG investors should love them.

ROIC is the North Star in analyzing a company's financial strength and future prospects. Diving into whether a company's ROIC is rising or falling is important, and understanding the drivers behind its ROIC trend line will make you a better investor. The DuPont Analysis breaks ROIC down further to see what's driving returns: high profit margin or high invested-capital turnover.

As part of identifying the drivers, we study a company's business model and its competitive advantages (or its competitive edge), and examine whether these edges are getting sharper or deteriorating. We also study the competitive environment, market-share trends, the rationality (or lack thereof) of pricing in the industry, and barriers to entry. We also employ frameworks like Porter's Five Forces to help with measuring the "moat," or competitive advantage, enjoyed by a company.

Next, we take a look at the net operating profit after tax (NOPAT) -- a crucial number used to calculate FCF, which also serves as the numerator in the ROIC equation; the company's margin profile; and the amount of operating leverage inherent in the business model. We previously demonstrated how to analyze the balance sheet and how efficiently it's managed, because invested capital is the denominator in the ROIC equation.

We also analyze management's ability to allocate capital, the performance metrics used to incentivize management, management's understanding of intrinsic value growth, and the language the company uses when speaking about growing business and shareholder value. Connecting the dots between revenue, ROIC, and FCF is a hugely positive sign.

Lastly, we analyze whether management incorporates ESG into everything it does, because ample research shows that focusing on ESG improves ROIC. Organic revenue growth and high ROIC are ultimately what create the FCF growth (particularly FCF-per-share growth) that we so desire. Remember that FCF is the amount of cash that is available to investors, as it's the amount of surplus cash generated by a business after spending capital (investing) to maintain and grow its assets. This FCF can be returned to shareholders as dividends, share buybacks, or by paying down debt. (If the company paid down all debt, there would be no more creditors to have a claim on cash flow, so equity holders would have full claim -- after the government, which gets its take through taxes. So by paying down debt, management is returning capital to equity holders.) The free cash can also be used to make acquisitions, or build the cash on the balance sheet.

For more on FCF, read Free Cash Flow and Shareholder Yield by Bill Priest and Lindsay McClelland, Free Cash Flow by George Christy, or Creative Cash Flow Reporting by Charles Mulford and Eugene Comiskey.

# Imports

In [26]:
# Necessary imports
import FundamentalAnalysis as fa
import pandas as pd
pd.options.display.max_rows = 999
import requests
from bs4 import BeautifulSoup

# Getting Company Ticker Symbols

In [31]:
# Getting site
page = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BeautifulSoup(page.content, 'html.parser')

# Finding table
table = soup.find(id='constituents').tbody

# Getting table rows
table_rows = table.find_all('tr')

# Finding company ticker in row
companies = []
for row in table_rows[1:70]:
    elements = row.find_all('td')
    company = elements[0].text
    company = company.replace('\n','')
    companies.append(company)

# Getting Financial Data

For each company of interest, the following metrics will be generated:
- roic
- roic growth
- fcf per share
- fcf per share growth

In [32]:
# Getting api key from local files
api_key = open('/Users/MichaelWirtz/Desktop/pathfile/fundamental_analysis_api_key.txt').read()

In [39]:
def fa_metrics(companies, num_years):

    # Make empty data frame
    fcf_roic = pd.DataFrame(columns=['company',
                                     'fcf_per_share_growth',
                                     'roic_growth',
                                     'current_roic',
                                     'current_fcf',
                                     'combined_growth_avg'])
    
    # Looping through companies
    for company in companies:
        # Get key metrics
        company_metrics = fa.key_metrics(company,api_key, 'annual')
        # Get current roic and fcf per share
        current_roic = company_metrics.loc['roic'][0]
        current_fcf = company_metrics.loc['freeCashFlowPerShare'][0]
        
        # Defining desired metrics
        metrics = ['freeCashFlowPerShare','roic']
        fcf_per_share_growth = 'NaN'
        roic_growth = 'NaN'
        
        # Get average growth metrics
        for metric in metrics:
            listed_metrics = company_metrics.loc[metric][0:num_years].to_list()
            # Make list of growth rates for each year
            annual_growth = []
            for i in range(0,(len(listed_metrics) -1)):
                yoy_g = (listed_metrics[i] / listed_metrics[i+1]) - 1
                annual_growth.append(yoy_g)
            # Get average growth over specified period
            total_period_growth_rate = sum(annual_growth) / len(annual_growth)
            # Assign value to correct metric
            if metric == 'freeCashFlowPerShare':
                fcf_per_share_growth = total_period_growth_rate
            elif metric == 'roic':
                roic_growth = total_period_growth_rate
        
        # Combining growth rates
        combined_growth_avg = (fcf_per_share_growth + roic_growth) / 2
            
        # Appending metrics to data frame for each company
        fcf_roic = fcf_roic.append({'company': company,
                                    'fcf_per_share_growth': fcf_per_share_growth,
                                    'roic_growth': roic_growth,
                                    'current_roic': current_roic,
                                    'current_fcf': current_fcf,
                                    'combined_growth_avg': combined_growth_avg}, ignore_index=True)
        
    # Sorting values in data frame
    fcf_roic = fcf_roic.sort_values(by=['combined_growth_avg'], ascending=False)
    
    # Return total average growth over specified period
    return fcf_roic  

In [40]:
df = fa_metrics(companies,5)

ValueError: Limit Reach . Please upgrade your plan or visit our documentation for more details at https://financialmodelingprep.com/developer/docs/pricing 

In [38]:
df

,company,fcf_per_share_growth,roic_growth,current_roic,current_fcf
54,ADSK,-1.532257,7.638189,0.099591,6.198908
65,BRK.B,0.169978,4.952965,0.128752,13897.644108
24,MO,0.277372,1.797123,0.179656,4.388633
9,AES,2.946402,1.473751,0.016689,1.301087
67,BIO,-3.646240,1.354692,0.318311,12.041852
17,ALXN,1.947733,1.163030,0.034530,13.158564
60,BLL,-0.885294,0.856915,0.049754,0.963449
29,AEP,0.130545,0.838524,0.027972,-3.795061
51,AIZ,2.571533,0.816409,0.072392,21.037093
47,APTV,-0.024175,0.334711,0.119724,3.282582
